In [ ]:
%pip install numpy scikit-learn
%pip install --no-cache-dir --force-reinstall https://dm.cs.tu-dortmund.de/nats/nats25_05_05_bonus_collapsed_gibbs_sampler-0.1-py3-none-any.whl
import nats25_05_05_bonus_collapsed_gibbs_sampler

# Implement LDA with Gibbs Sampling

In this *bonus* assignment, your task is to implement LDA yourself using Gibbs sampling, without using libraries such as Gensim or sklearn.

In [ ]:
### Load the input data - do not modify
import json, gzip, urllib, numpy as np
file_path, _ = urllib.request.urlretrieve("https://dm.cs.tu-dortmund.de/nats/data/minecraft-articles.json.gz")
raw = json.load(gzip.open(file_path, "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["heuristic"] for x in raw]
# To speed up processing times, we will make use of numba.
# Yet, there is no numba port for WebAssembly yet, so if
# you are running this on Pyodide/JupyterLite, we need to
# replace the jit decorator with a no-op stub.
if not "pyodide" in globals():
    %pip install numpy==2.1.3 # currently, numba does not support 2.2
    %pip install numba
    from numba import jit
else:
    # Use no-op stub.
    def jit(*args, **kwargs): return lambda function: function

In [ ]:
### Vectorize the text - do not modify
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words="english", min_df=5)
counts = cvect.fit_transform(texts)
vocabulary = cvect.get_feature_names_out()

In [ ]:
# Understand how the non-zero values are stored / accessible - read the docs!
print(".shape", counts.shape)
print(".data shape:", counts.data.shape)
print(".indices shape:", counts.indices.shape)
print(".indptr shape:", counts.indptr.shape)
# work on these data structures directly, avoid any operation that copies data such as .nonzero()!

## Initial random labeling

LDA begins with a random labeling of the tokens. For simplicity, we ignore that words can occur multiple times in a document, and always label them the same way. We want to store our labels in a dense data structure for efficiency!

In [ ]:
def initial_labeling(counts, num_topics, rng):
    """Generate a uniform random labeling of the desired shape"""
    pass # Your solution here

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_6_0(counts, initial_labeling)

## Initial statistics

Compute the initial count statistics of the labeling necessary for the Collapsed Gibbs Sampler

In [ ]:
@jit(nopython=True) # compile, otherwise this may be way too slow
def initial_statistics(indices, indptr, labels, num_docs, num_words, num_topics):
    pass # Your solution here
    return doc_topic, topic_word, topic # counts

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_9_0(counts, initial_labeling, initial_statistics)


## Collapsed Gibbs Sampler

Implement one step (one pass over the data set) of the collapsed Gibbs sampler.

We still ignore if words occur multiple times for simplicity.

You may (and will need to) modify the data structures in-place, because this is a Markov process.
Remove the label, sample a new label, add new label, repeat.

Because numba does not support current numpy random generator objects, nor weighted random choice, we need to implement our own helper first. But we also need to use compilation: with pure python one pass easily takes 45 seconds as opposed to 250 ms, i.e., the compiled code runs 180x faster.

In [ ]:
@jit(nopython=True) # compile, otherwise this may be way too slow
def weighted_random(p):
    """Choose an index of p randomly, weighted by values in p (which must sum to 1)"""
    pass # Your solution here
    # Hint: add a fallback for the (rare) case that the sum is slightly less than 1:
    return int(np.floor(np.random.rand() * p.shape[0]))

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_12_0(weighted_random)

In [ ]:
@jit(nopython=True) # compile, otherwise this may be way too slow
def gibbs(alpha, eta, indices, indptr, labels, doc_topic, topic_word, topic):
    num_topics = topic.shape[0]
    pass # Your solution here
    return labels # same data structure as above, for convenience

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_14_0(counts, gibbs, initial_labeling, initial_statistics)

## Implement LDA with Gibbs sampling

Write the outer loop of LDA. We will use a burn-in of 50 iterations, then aggregate 50 subsequent iterations to obtain the summary statistics. At 100 iterations, 250 ms above will likely be a tolerable 25 seconds.

In [ ]:
# no jit: the outer loop should be fine with just numpy + calls to the compiled functions above
def lda(counts, k, alpha, eta, rng, burnin=50, measure=50, every=1):
    """Latent Dirichlet Allocation. Return the factors and document assignment"""
    np.random.seed(rng.integers(0x7FFFFFF)) # needed for numba
    pass # Your solution here
    return factors, assignment

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_17_0(counts, lda)


## Explore your result

Explore the result: write a function to determine the most important words for each factor, and the most relevant documents.

**COPY your code from the first file here** (one of the rare cases, where copying is okay)

In [ ]:
def most_important(vocabulary, factor, k=10):
    """Most important words for each factor"""
    pass # Your solution here

def most_relevant(assignment, k=5):
    """Most relevant documents for each factor (return document indexes)"""
    pass # Your solution here

def explain(vocabulary, titles, classes, factors, assignment, weights=None):
    """Print an explanation for each factor.
       If weights is None, use the relative share of the assignment weights.
       Print the ARI when assigning each document to its maximum only."""
    from sklearn.metrics import adjusted_rand_score
    pass # Your solution here

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_20_0(counts, lda)

In [ ]:
# Explore your result (reduce the number of iterations prior to validating/submitting, to not get a timeout)
rng = np.random.default_rng(0)
%time lda_factors, lda_assignment = lda(counts, 8, 1/8, 1/8, rng, 50, 50)
explain(vocabulary, titles, classes, lda_factors, lda_assignment)

In [ ]:
nats25_05_05_bonus_collapsed_gibbs_sampler.hidden_tests_22_0(most_important, lda_factors, vocabulary)